In [71]:
# config
api_key = ""


In [ ]:
# bibs
import requests
import pandas as pd
import os

In [86]:

def is_inside_bh(latitude, longitude):
    # Solicitação à Geocoding API para obter o endereço correspondente às coordenadas
    url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    # Verifique se o resultado contém informações sobre Belo Horizonte
    inside = False
    for result in data.get('results', []):
        for address_component in result.get('address_components', []):
            if 'Belo Horizonte' in address_component.get('long_name', ''):
                inside = True
                break
    # if inside:
    #     print("O ponto está dentro da cidade de Belo Horizonte.")
    # else:
    #     print("O ponto não está dentro da cidade de Belo Horizonte.")

    return inside


In [87]:

# Diretório contendo os arquivos CSV
directory = ['estacoes_automaticas', "estacoes_convencionais"]

# Inicialize uma lista para armazenar todos os DataFrames
dfs = []

for dir in directory:
    # Lista os arquivos no diretório
    for filename in os.listdir(dir):
        if filename.endswith('.csv'):
            file_path = os.path.join(dir, filename)
            
            # Inicialize variáveis para armazenar metadados
            metadata = {}

            # Leia o arquivo linha por linha para extrair metadados
            with open(file_path, 'r') as file:
                for line in file:
                    if line.strip() and len(metadata) < 8:  # Limitar a extração de metadados a 8 linhas
                        key, value = line.strip().split(': ')
                        metadata[key] = value

            latitude = metadata["Latitude"]
            longitude = metadata["Longitude"]

            if is_inside_bh(latitude, longitude):
                # Leia a parte de dados do arquivo em um DataFrame
                df = pd.read_csv(file_path, skiprows=10, delimiter=';', parse_dates=['Data Medicao'], decimal=',')
                df = df.drop('Unnamed: 2', axis=1)

                # Crie um DataFrame para os metadados
                metadata_df = pd.DataFrame.from_dict(metadata, orient='index').T

                # Crie um novo DataFrame com os metadados repetidos para todas as linhas
                metadata_df = pd.concat([metadata_df] * len(df), ignore_index=True)

                # Concatene o DataFrame de metadados com o DataFrame de dados
                df = pd.concat([metadata_df, df], axis=1)

                # Adicione o DataFrame resultante à lista de DataFrames
                dfs.append(df)

    # Concatene todos os DataFrames em um único DataFrame final
    final_df = pd.concat(dfs, ignore_index=True)
    final_df.to_csv(f'dados_{dir}_bh.csv', index=False)


In [84]:
final_df

,Nome,Codigo Estacao,Latitude,Longitude,Altitude,Situacao,Data Inicial,Data Final,Data Medicao,"PRECIPITACAO TOTAL, DIARIO (AUT)(mm)"
0,BELO HORIZONTE (PAMPULHA),A521,-19.88388888,-43.96944443,854.02,Operante,2006-10-09,2023-01-01,2006-10-09,NaN
1,BELO HORIZONTE (PAMPULHA),A521,-19.88388888,-43.96944443,854.02,Operante,2006-10-09,2023-01-01,2006-10-10,NaN
2,BELO HORIZONTE (PAMPULHA),A521,-19.88388888,-43.96944443,854.02,Operante,2006-10-09,2023-01-01,2006-10-11,0.4
3,BELO HORIZONTE (PAMPULHA),A521,-19.88388888,-43.96944443,854.02,Operante,2006-10-09,2023-01-01,2006-10-12,0.0
4,BELO HORIZONTE (PAMPULHA),A521,-19.88388888,-43.96944443,854.02,Operante,2006-10-09,2023-01-01,2006-10-13,2.6
...,...,...,...,...,...,...,...,...,...,...
14542,BELO HORIZONTE - CERCADINHO,F501,-19.97999999,-43.95861111,1199.55,Operante,2013-12-26,2023-01-01,2022-12-28,0.8
14543,BELO HORIZONTE - CERCADINHO,F501,-19.97999999,-43.95861111,1199.55,Operante,2013-12-26,2023-01-01,2022-12-29,0.0
14544,BELO HORIZONTE - CERCADINHO,F501,-19.97999999,-43.95861111,1199.55,Operante,2013-12-26,2023-01-01,2022-12-30,0.0
14545,BELO HORIZONTE - CERCADINHO,F501,-19.97999999,-43.95861111,1199.55,Operante,2013-12-26,2023-01-01,2022-12-31,71.4
